In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
from pathlib import Path
import numpy as np
import multiprocessing

In [4]:
# The optical flow input will look like this
of = torch.randn(3,2,640,480).cuda()

In [2]:
model = EfficientNet.from_pretrained("efficientnet-b0", in_channels=2, num_classes=1).cuda()

Loaded pretrained weights for efficientnet-b0


In [6]:
features = model(of)
features.shape

torch.Size([3, 1])

In [3]:
class OFDataset(Dataset):
    def __init__(self, of_dir, label_f):
        self.len = len(list(Path(of_dir).glob('*.npy')))
        self.of_dir = of_dir
        self.label_file = open(label_f).readlines()
    def __len__(self): return self.len
    def __getitem__(self, idx):
        of_array = np.load(Path(self.of_dir)/f'{idx}.npy')
        of_tensor = torch.squeeze(torch.Tensor(of_array))
        label = float(self.label_file[idx].split()[0])
        return [of_tensor, label]

In [4]:
ds = OFDataset('/home/sharif/Documents/RAFT/train_predictions', '/home/sharif/Documents/commai-challenge/data/labels/train.txt')

In [5]:
lengths = [int(len(ds)*0.8)+1, int(len(ds)*0.2)]
train_ds, val_ds = torch.utils.data.random_split(ds, lengths)

In [6]:
sample = ds[3]
assert type(sample[0]) == torch.Tensor
assert type(sample[1]) == float

In [7]:
cpu_cores = multiprocessing.cpu_count()
cpu_cores

6

In [8]:
train_dl = DataLoader(train_ds, batch_size=6, shuffle=True, num_workers=cpu_cores)
val_dl = DataLoader(val_ds, batch_size=6, shuffle=True, num_workers=cpu_cores)

In [9]:
epochs = 10
train_steps = 25
val_steps = 100

In [10]:
criterion = nn.MSELoss()
opt = optim.Adam(model.parameters())

In [11]:
writer = SummaryWriter('runs/optical_flow_exp_7')

In [12]:
n_validations = 0
validation_losses = []

for epoch in range(epochs):
    
    running_loss = 0.0
    for i, sample in enumerate(train_dl):
        
        of_tensor = sample[0].cuda()
        label = sample[1].float().cuda()
        opt.zero_grad()
        pred = torch.squeeze(model(of_tensor))
        loss = criterion(pred, label)
        
        loss.backward()
        opt.step()
        
        running_loss += loss.item()
        if i % train_steps == 0: 
            writer.add_scalar('training loss',
                             running_loss / train_steps,
                             epoch * len(train_dl) + i)
            running_loss = 0.0
            
        if i % 100 == 0: print(f'{i}/{len(train_dl)}')
        
        # val loss
        if i % 1000 == 0:
            with torch.no_grad():
                for j, val_sample in enumerate(val_dl):
                    of_tensor = val_sample[0].cuda()
                    label = val_sample[1].float().cuda()
                    pred = torch.squeeze(model(of_tensor))
                    loss = criterion(pred, label)

                    running_loss += loss.item()
                    if j % 25 == 0: print(f'{j}/{len(val_dl)}')

                    if j == val_steps:
                        l = running_loss / val_steps
                        writer.add_scalar('validation loss',
                                         l,
                                         n_validations)
                        validation_losses.append(l)
                        if min(validation_losses) == l:
                            p = f'/home/sharif/Documents/commai-challenge/trained_models/{epoch}_{i}.pth'
                            torch.save(model.state_dict(), p)
                        
                        running_loss = 0.0
                        n_validations += 1
                        break

0/2720
0/680
25/680
50/680
75/680
100/680
100/2720
200/2720
300/2720
400/2720
500/2720
600/2720
700/2720
800/2720
900/2720
1000/2720
0/680
25/680
50/680
75/680
100/680
1100/2720
1200/2720
1300/2720
1400/2720
1500/2720
1600/2720
1700/2720
1800/2720
1900/2720
2000/2720
0/680
25/680
50/680
75/680
100/680
2100/2720
2200/2720
2300/2720
2400/2720
2500/2720
2600/2720
2700/2720
0/2720
0/680
25/680
50/680
75/680
100/680
100/2720
200/2720
300/2720
400/2720
500/2720
600/2720
700/2720


KeyboardInterrupt: 